In [ ]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

cpu


In [ ]:
!pip install opencv-python==4.4.0.46

     |████████████████████████████████| 49.5MB 154kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30


In [ ]:
!pip install optuna

     |████████████████████████████████| 296kB 16.0MB/s 
     |████████████████████████████████| 163kB 40.5MB/s 
     |████████████████████████████████| 81kB 8.3MB/s 
     |████████████████████████████████| 81kB 8.4MB/s 
     |████████████████████████████████| 51kB 5.3MB/s 
     |████████████████████████████████| 143kB 28.4MB/s 
     |████████████████████████████████| 112kB 52.0MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp37-none-any.whl size=11107 sha256=0a02576b9927797bd5da877d668a9ccbba60c9a7f7d6d6911397c42fdb48c7fa
  Stored in directory: /root/.cache/pip/wheels/25/af/b8/3407109267803f4015e1ee2ff23be0c8c19ce4008665931ee1
Successfully built pyperclip


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os


GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab Notebooks/Computer Vision/Coursework/' 
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

['CW_Dataset-2.zip', 'Untitled0.ipynb', 'EmotionRecognition.py', '__pycache__']


In [ ]:
### FROM LAB 9 ### 
# Identify path to zipped dataset
zip_path = os.path.join(GOOGLE_DRIVE_PATH, 'CW_Dataset-2.zip')

# Copy it to Colab
!cp '{zip_path}' .

# Unzip it (removing useless files stored in the zip)
!yes|unzip -q CW_Dataset-2.zip

# Delete zipped version from Colab (not from Drive)
!rm CW_Dataset-2.zip

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import cv2
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor, Lambda
import time
import pandas as pd
import copy
from PIL import Image
from skimage import io, img_as_ubyte
from skimage.feature import hog
from skimage import data, exposure
from sklearn.model_selection import train_test_split
import optuna
from optuna.trial import TrialState
%matplotlib inline

In [ ]:
class CustomImageDataset(Dataset):

    def __init__(self, annotations_df, img_dir, transform=None, target_transform=None):

        self.img_labels = annotations_df
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = io.imread(img_path)
        h = image.copy()

        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)

      
        sample = {"image": image, "label": label}
        return image, label

In [ ]:
class MLP(nn.Module):
    def __init__(
            self,
            hidden_num = 40,
            dropout = 0.1,
            nonlin = torch.nn.Sigmoid()

    
    ):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(70,hidden_num)
        self.fc2 = nn.Linear(hidden_num,8)
        self.non_lin = nonlin
        self.dropout = dropout
        
    def forward(self, x):
        hidden = self.fc1(x)
        hidden = self.non_lin(hidden)
        output = self.fc2(hidden)
        return output
#OPTIMAL PARAMETERS 

In [ ]:
data_dir = "CW_Dataset-2"
l = os.path.join(data_dir, "labels", "list_label_train.txt")
l_test = os.path.join(data_dir, "labels", "list_label_test.txt")
d = os.path.join(data_dir, "train")
d_test = os.path.join(data_dir, "test")

In [ ]:
lab = pd.read_csv(l, delimiter= ' ',header=None)
lab = lab.rename(columns={0: "name", 1: "lbl"})
lab['name'] = lab['name'].str[:-4] + "_aligned" + ".jpg"
lab['lbl'] = lab['lbl']-1

train_lab, val_lab = train_test_split(lab, test_size=0.2) #Split the dataset to get a set for validation

test_lab = pd.read_csv(l_test, delimiter= ' ',header=None)
test_lab = test_lab.rename(columns={0: "name", 1: "lbl"})
test_lab['name'] = test_lab['name'].str[:-4] + "_aligned" + ".jpg"
test_lab['lbl'] = test_lab['lbl'] - 1

In [ ]:
data_means = [0.485, 0.456, 0.406]
data_stds = [0.229, 0.224, 0.225]

training_data = CustomImageDataset(train_lab, d, transform=transforms.Compose([transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomGrayscale(),
        transforms.RandomRotation(10),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(data_means, data_stds)]))

validation_data = CustomImageDataset(val_lab, d, transform=transforms.Compose([transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize(data_means, data_stds)]))

test_data = CustomImageDataset(test_lab, d_test, transform=transforms.Compose([transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize(data_means, data_stds)]))

In [ ]:
ds_train = DataLoader(training_data, batch_size =4, shuffle=True) 
ds_val = DataLoader(validation_data, batch_size =4, shuffle=True) 
ds_test = DataLoader(test_data, batch_size =4, shuffle=False) 

In [ ]:
dataset_sizes = len(training_data.img_labels)

In [ ]:
def sift_conv(ds, b_num):
  des_list = []
  l_set = []
  for image, label in ds: #taking images and applying sift 
    for i in range(b_num-1):
      #print(image[i].shape)
      inp = image[i].numpy().transpose((1, 2, 0))
      image8bit = cv2.normalize(inp, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
      sift = cv2.SIFT_create()
      SIFT_kp, SIFT_des = sift.detectAndCompute(image8bit, None)
      if SIFT_des is not None:
          des_list.append(SIFT_des)
          l_set.append(label[i])
  des_array = np.vstack(des_list)
  return des_array, des_list, l_set

In [ ]:
train_arr, train_list, train_labels = sift_conv(ds_train, 4)
val_arr, val_list, val_labels = sift_conv(ds_val, 4)
test_arr, test_list, test_labels = sift_conv(ds_test, 4)


In [ ]:
from sklearn.cluster import MiniBatchKMeans
def kmeans_bovw(des_array, des_list):
  batch_size = des_array.shape[0] // 4
  kmeans = MiniBatchKMeans(n_clusters=70, batch_size=batch_size).fit(des_array) #Doing KMeans for BoVW

  # Convert descriptors into histograms of codewords for each image
  hist_list = []
  idx_list = []

  for des in des_list:
      hist = np.zeros(70)

      idx = kmeans.predict(des)
      idx_list.append(idx)
      for j in idx:
          hist[j] = hist[j] + (1 / len(des))
      hist_list.append(hist)

  hist_array = np.vstack(hist_list)
  return hist_array

In [ ]:
train_bovw = kmeans_bovw(train_arr, train_list)
val_bovw = kmeans_bovw(val_arr, val_list)
test_bovw = kmeans_bovw(test_arr, test_list)

In [ ]:
from torch.utils.data import TensorDataset
train_ds = TensorDataset(torch.tensor(train_bovw), torch.tensor(train_labels))
val_ds = TensorDataset(torch.tensor(val_bovw), torch.tensor(val_labels))

train_dl = DataLoader(train_ds, batch_size=4)
val_dl = DataLoader(val_ds, batch_size = 4)

In [ ]:
training_dataset_sizes = len(train_labels)
validation_dataset_sizes = len(val_labels)

In [ ]:
inp, lab = next(iter(train_ds))
print(inp)
print(lab)

tensor([0.0000, 0.0000, 0.0222, 0.0222, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0222, 0.0000, 0.0222, 0.0000, 0.0000,
        0.0222, 0.0000, 0.0000, 0.0222, 0.0000, 0.0222, 0.0000, 0.0222, 0.0222,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0222, 0.0000, 0.0222,
        0.0000, 0.0444, 0.0000, 0.0444, 0.0000, 0.0000, 0.0000, 0.0444, 0.0000,
        0.0444, 0.0667, 0.0000, 0.0667, 0.0000, 0.0000, 0.0000, 0.0667, 0.0000,
        0.0000, 0.0667, 0.0222, 0.0444, 0.0222, 0.0222, 0.0000, 0.0444, 0.0000,
        0.0000, 0.0000, 0.0889, 0.0000, 0.0222, 0.0222, 0.0222],
       dtype=torch.float64)
tensor(4)


In [ ]:
### ADAPTED FROM LAB 9, FOLLOWEDD THIS TUTORIAL FOR OPTUNA https://broutonlab.com/blog/efficient-hyperparameter-optimization-with-optuna-framework

def train_model(trial, model, criterion, optimizer, num_epochs=10):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        
       # print(f"Epoch {epoch}/{num_epochs - 1}")
       # print('----------------')

        # Each epoch has a training and validation phase
        
        model.train()  # Set model to training mode
        running_loss = 0.0
        running_corrects = 0

            # Iterate over data
        for inputs, labels in train_dl:
       
                # move data to GPU
                input = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(True):
           
                    outputs = model(input.float())
                    _, preds = torch.max(outputs, 1)
      
                    
                    loss = criterion(outputs, labels)
             
                    # backward + optimize only if in training phase

                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.item() * input.size(0)
                running_corrects += torch.sum(preds == labels.data)

            # update learning rate with scheduler


        epoch_loss = running_loss / training_dataset_sizes
        epoch_acc = running_corrects.double() / training_dataset_sizes

        #print(f"train loss: {epoch_loss:.4f} train acc: {epoch_acc:.4f}")
      
        
        model.eval()  # Set model to evaluation mode
        running_loss = 0.0
        running_corrects = 0

        for inputs_val, labels_val, in val_dl:
                # move data to GPU
                input_val = inputs_val.to(device)
                labels_val = labels_val.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(False):
                    outputs_val = model(input_val.float())
               
                    _, preds_val = torch.max(outputs_val, 1)
                    loss_val = criterion(outputs_val, labels_val)

                # statistics
                running_loss += loss_val.item() * input_val.size(0)
                running_corrects += torch.sum(preds_val == labels_val.data)


        epoch_loss = running_loss / validation_dataset_sizes
        epoch_acc = running_corrects.double() / validation_dataset_sizes


            # deep copy the model with best accuracy on validation set
        if epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())


        trial.report(epoch_acc, epoch)
        if trial.should_prune():
            raise optuna.TrialPruned()

    time_elapsed = time.time() - since


    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, best_acc

In [ ]:
def objective(trial):
    
    # Hyperparameters
    params = {
       # "model_name": trial.suggest_categorical('model_name',["resnet18", "alexnet", "vgg16"]),
        "lr": trial.suggest_loguniform('lr', 1e-4, 1e-2),
        "optimizer_name": trial.suggest_categorical('optimizer_name',["SGD", "Adam"])
    }
    
    # Get pretrained model
    model = MLP()

    model = model.to(device)
    
    # Define criterion
    criterion = nn.CrossEntropyLoss()
    
    # optimizer params
    optimizer = getattr(
        torch.optim, params["optimizer_name"]
    )(model.parameters(), lr=params["lr"])
    
    # Train model
    best_model, best_acc = train_model(trial, model, criterion, optimizer, num_epochs=20)

    return best_acc

In [ ]:
sampler = optuna.samplers.TPESampler()    
study = optuna.create_study(
    sampler=sampler,
    pruner=optuna.pruners.HyperbandPruner(
    ),
    direction='maximize')
study.optimize(func=objective, timeout=2000)

[I 2021-04-30 01:23:49,941] A new study created in memory with name: no-name-db955727-ab49-4683-940b-ec10a9191a3a
[I 2021-04-30 01:24:17,265] Trial 0 finished with value: 0.3883758826724606 and parameters: {'lr': 0.0012486332196996565, 'optimizer_name': 'Adam'}. Best is trial 0 with value: 0.3883758826724606.
[I 2021-04-30 01:24:37,998] Trial 1 finished with value: 0.3872895165670831 and parameters: {'lr': 0.00010450526966266773, 'optimizer_name': 'SGD'}. Best is trial 0 with value: 0.3883758826724606.
[I 2021-04-30 01:24:58,728] Trial 2 finished with value: 0.3872895165670831 and parameters: {'lr': 0.0009462975385175645, 'optimizer_name': 'SGD'}. Best is trial 0 with value: 0.3883758826724606.
[I 2021-04-30 01:25:19,536] Trial 3 finished with value: 0.3872895165670831 and parameters: {'lr': 0.0011347494951689655, 'optimizer_name': 'SGD'}. Best is trial 0 with value: 0.3883758826724606.
[I 2021-04-30 01:25:40,266] Trial 4 finished with value: 0.3872895165670831 and parameters: {'lr': 0